**trainer**

In [ ]:
import cv2
import os
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display
from threading import Thread, Lock
import time
import numpy as np
import dlib
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [ ]:
#RTSP_URL = "jouwRTSP-stream"
#cap = cv2.VideoCapture(1) #of 0

In [ ]:
BASE_DIR = "../dataset/" # of je eigen locatie
os.makedirs(BASE_DIR, exist_ok=True)

### model trainer met LBPH (Local Binary Pattern Histogram)

In [ ]:
# Ophalen van beschikbare dataset-mappen
available_datasets = [d for d in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR, d))]

# Dropdown-menu voor dataset-selectie
dataset_dropdown = widgets.Dropdown(
    options=available_datasets,
    description="Selecteer Naam:"
)
train_button = widgets.Button(description="Train Model")
output = widgets.Output()

# Weergave in Notebook
display(dataset_dropdown, train_button, output)

# **Functie om een afbeelding te verbeteren**
def preprocess_image(img):
    """ Zorgt voor betere herkenning door bewerking van afbeeldingen """
    img = cv2.equalizeHist(img)  # Helderheid normaliseren
    img = cv2.GaussianBlur(img, (3, 3), 0)  # Ruis verminderen
    return img

# **Data augmentatie toepassen**
def augment_image(img):
    """ Voegt variatie toe aan trainingsdata """
    augmented_images = []

    # Oorspronkelijke afbeelding
    augmented_images.append(img)

    # Horizontale spiegeling
    augmented_images.append(cv2.flip(img, 1))

    # Kleine rotaties
    for angle in [-10, 10]:  
        rows, cols = img.shape
        M = cv2.getRotationMatrix2D((cols/2, rows/2), angle, 1)
        rotated = cv2.warpAffine(img, M, (cols, rows))
        augmented_images.append(rotated)

    # Lichte contrastwijzigingen
    alpha = random.uniform(0.8, 1.2)  # Contrast (0.8-1.2)
    beta = random.randint(-20, 20)  # Helderheid (-20 tot 20)
    contrast = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    augmented_images.append(contrast)

    return augmented_images

# **Train het model met betere data**
def train_model(_):
    selected_name = dataset_dropdown.value
    if not selected_name:
        with output:
            print("Selecteer een dataset om te trainen.")
        return

    PROCESSED_DIR = os.path.join(BASE_DIR, selected_name, "processed_faces")
    MODEL_PATH = os.path.join(BASE_DIR, selected_name, f"face_model_{selected_name}.yml")

    # Controleren of er gezichten beschikbaar zijn
    image_files = [f for f in os.listdir(PROCESSED_DIR) if f.endswith(".jpg")]
    if not image_files:
        with output:
            print(f"Geen gezichten gevonden in {PROCESSED_DIR}, training geannuleerd.")
        return

    # Gegevens en labels verzamelen
    images = []
    labels = []
    label_dict = {name: idx for idx, name in enumerate(available_datasets)}  # Unieke labels per persoon

    for filename in image_files:
        img_path = os.path.join(PROCESSED_DIR, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            with output:
                print(f"Kan afbeelding {filename} niet laden, overslaan.")
            continue

        # **Verbeter de afbeelding en pas data augmentation toe**
        img = preprocess_image(img)
        augmented_imgs = augment_image(img)

        for aug_img in augmented_imgs:
            aug_img_resized = cv2.resize(aug_img, (200, 200))
            images.append(aug_img_resized)
            labels.append(label_dict[selected_name])

    # **Check de datasetgrootte**
    print(f"Totaal {len(images)} beelden na augmentatie.")

    # **Model trainen**
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(np.array(images, dtype=np.uint8), np.array(labels, dtype=np.int32))

    # **Opslaan in `datasets/<naam>/`**
    model.save(MODEL_PATH)

    with output:
        print(f"Model getraind en opgeslagen als {MODEL_PATH} met {len(images)} beelden.")

# Koppel knop aan functie
train_button.on_click(train_model)
